In [4]:
# Imports
from pandas import read_csv 
from sklearn.model_selection import train_test_split

from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score

In [10]:
#  Load dataset and create train-test sets
data = read_csv('data/hairdryer.csv')
var_names = data.columns[1:-1]


In [12]:
data = data.to_numpy()
scaler = MinMaxScaler()
data = scaler.fit_transform(data)


In [14]:
# train/test split
X = data[:,0:-1]
y = data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# %% Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)  # Number of clusters
clust_centers, part_matrix, _ = cl.cluster(method='fcm')

In [16]:
# %% Estimate membership functions parameters
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()

In [17]:
# %% Estimate consequent parameters
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms()

In [18]:
# %% Build first-order Takagi-Sugeno model
modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
model = modbuilder.get_model()

 * Detected 10 rules / clusters
 * Detected Sugeno model type


In [19]:
# %% Get model predictions
modtester = SugenoFISTester(model, X_test, var_names)
y_pred = modtester.predict()[0]

AttributeError: 'str' object has no attribute 'evaluate'